## Simple Latent Factor-Based Recomender

Here I'll use gradient descent to implement a machine-learning-based recommender (a latent-factor model).
First, we build some utility data structures to store the variables of our model (alpha, userBiases, and itemBiases)

## Part 1: Setting up the Data

This dataset is a series of reviews and ratings of Digital Video Games from Amazon. 
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz

In [1]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
import numpy
import random

path = "./amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"

I am going to set up the data in the same way I did in Similarity-Based Recommendation Systems. Based on Ratings and Votes I will make recomendations based on a selected Game.

In [2]:
f = gzip.open(path, 'rt', encoding="utf8")

header = f.readline()
header = header.strip().split('\t')

dataset = []

for line in f:
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    dataset.append(d)

In [4]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

for d in dataset:
    user,item = d['customer_id'], d['product_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [5]:
#Getting the respective lengths of our dataset and dictionaries
N = len(dataset)
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)

#Getting a list of keys
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())

#This is equivalent to our Rating Mean from week 1
alpha = sum([d['star_rating'] for d in dataset]) / len(dataset)

#Create another two defaultdict's, this time being float types because they are prediction based
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

#Can't forget our MSE function
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

The actual prediction function of the model is simple: Just predict using a global offset (alpha), a user offset (beta_u in the slides), and an item offset (beta_i)

In [6]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

I'll use another library in this notebook to perform gradient descent. This library requires that I pass it a "flat" parameter vector (theta) containing all of our parameters. This utility function just converts between a flat feature vector, and my model parameters, i.e., it "unpacks" theta into our offset and bias parameters.

In [7]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

The "cost" function is the function I am trying to optimize. Again this is a requirement of the gradient descent library I'll use. In this case, I'm just computing the (regularized) MSE of a particular solution (theta), and returning the cost.

In [8]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d['customer_id'], d['product_id']) for d in dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

The derivative function is the most difficult to implement, but follows the definitions of the derivatives for this model. **This step could be avoided if using a gradient descent implementation based on (e.g.) Tensorflow.**

In [9]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in dataset:
        u,i = d['customer_id'], d['product_id']
        pred = prediction(u, i)
        diff = pred - d['star_rating']
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return numpy.array(dtheta)

Compute the MSE of a trivial baseline (always predicting the mean) for comparison:

In [10]:
alwaysPredictMean = [alpha for d in dataset]
labels = [d['star_rating'] for d in dataset]

MSE(alwaysPredictMean, labels) 

2.371535478415058

Finally, I can run gradient descent. This particular gradient descent library takes as inputs (1) the cost function (implemented above); (2) Initial parameter values; (3) Our derivative function; and (4) Any additional arguments to be passed to the cost function (in this case the labels and the regularization strength).

In [11]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, 0.001))

MSE = 2.371535478415058
MSE = 2.24524147757418
MSE = 3.5132167887118895
MSE = 2.211284736177427
MSE = 2.1282724344997055
MSE = 2.1237846430131158
MSE = 2.1082323438773205
MSE = 2.0506621773917177
MSE = 2.0084242729426465
MSE = 1.9638605236073836
MSE = 1.949665969308186
MSE = 1.9526044752580993
MSE = 1.949979803237368
MSE = 1.947177520911931
MSE = 1.94167155956151
MSE = 1.9371723194880355
MSE = 1.9372094011180663
MSE = 1.9371172036970188
MSE = 1.9387128230498027
MSE = 1.9371058127164145
MSE = 1.9367718672265353
MSE = 1.936582679959965
MSE = 1.9364439086796212
MSE = 1.9364491895797546
MSE = 1.9363337231249247
MSE = 1.9363747332022736
MSE = 1.936336921329935
MSE = 1.9361886799270902
MSE = 1.9360294312579918
MSE = 1.9361349519494546
MSE = 1.936088241369815
MSE = 1.9360580364151272
MSE = 1.9360133449345616
MSE = 1.9360450849055035
MSE = 1.9360468678360734
MSE = 1.9360496055920227
MSE = 1.9360440361754412
MSE = 1.9360414212483186
MSE = 1.93604011774619


(array([ 3.67299564e+00, -3.01669105e-03,  4.11425344e-03, ...,
        -4.53449134e-03, -1.15367480e-02,  8.99700061e-03]),
 2.0190378453775235,
 {'grad': array([-6.09081944e-06, -6.16484242e-09,  1.82673586e-09, ...,
         -4.49900663e-10,  1.15862373e-08, -7.80519517e-09]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 39,
  'nit': 32,
  'warnflag': 0})

## Part 3: Complete Latent Factor Model



For each user and item I now have a low dimensional descriptor (which represents a user's preferences), of dimension K.

In [12]:
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
userGamma = {}
itemGamma = {}

K = 2

In [13]:
for u in reviewsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]
    
for i in reviewsPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

Again I must implement an "unpack" function. This is the same as before, though has some additional terms.

In [14]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:index+nUsers]))
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index += K

Similarly, the cost and derivative functions serve the same role as before, though their implementations are somewhat more complicated.

In [15]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])


def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])


def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d['customer_id'], d['product_id']) for d in dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in users:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost


def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in reviewsPerUser:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in reviewsPerItem:
        dItemGamma[i] = [0.0 for k in range(K)]
    for d in dataset:
        u,i = d['customer_id'], d['product_id']
        pred = prediction(u, i)
        diff = pred - d['star_rating']
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2*lamb*userGamma[u][k]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return numpy.array(dtheta)

Again I optimize using our gradient descent library, and compare to a simple baseline.

In [16]:
MSE(alwaysPredictMean, labels) #Same as our previous baseline for comparison

2.371535478415058

In [17]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
                                   [0.0]*(nUsers+nItems) + # Initialize beta
                                   [random.random() * 0.1 - 0.05 for k in range(K*(nUsers+nItems))], # Gamma
                             derivative, args = (labels, 0.001), maxfun = 10, maxiter = 10)

#Note the "maxfun = 10" and "maxiter = 10" this is because this function will go on for over 
# 20 iterations taking far too long to compute.

MSE = 2.4039801031213996
MSE = 2.867519452301098
MSE = 2.358391871307474
MSE = 2.3459250448933977
MSE = 2.2995554765387243
MSE = 2.1016330745347958
MSE = 2.0812032814769577
MSE = 1.990518796367288
MSE = 1.9578645004077628
MSE = 1.9281559542914537
MSE = 1.9287004135756038


(array([ 3.65769030e+00, -2.99562803e-03,  3.63512832e-03, ...,
         2.69287851e-03, -9.27493296e-04,  3.82260396e-03]),
 2.0278889937099915,
 {'grad': array([-8.98050015e-04,  2.56787120e-07, -2.69625815e-06, ...,
          5.40463286e-06, -1.92502415e-06,  7.59761697e-06]),
  'task': b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT',
  'funcalls': 11,
  'nit': 8,
  'warnflag': 1})